## Desarrollo del Ejercicio

Una vez completados los pasos previos, procedemos a desarrollar el ejercicio. Como indicamos en la sección anterior, esta parte del ejercicio la realizaremos sobre este **Notebook ya abierto en nuestro PC en Visual Studio Code**.

### Despliegue Aplicación Unity Catalog

En primer lugar, clonamos el repositorio de GitHub donde está publicado de forma abierta la actual versión de Unity Catalog. Seleccionamos en particular el *commit id* **5d668c164ec17aa28c9974c6c218afc06da91750** por ser el actual en el momento de realizar este tutorial. Esto creará en nuestra carpeta de trabajo una subcarpeta llamada *unitycatalog* que contiene el código abierto de la herramienta.

In [ ]:
%%sh
if [ ! -d "unitycatalog" ]; then
  git clone https://github.com/unitycatalog/unitycatalog.git
  cd unitycatalog
  git checkout 5d668c164ec17aa28c9974c6c218afc06da91750
else
  echo "La carpeta 'unitycatalog' ya existe."
fi

A continuación, utilizamos Docker Desktop para levantar la aplicación Unity Catalog. La definición de los contenedores que componen esta aplicación la podemos encontrar en el fichero *compose.yaml*, descargado al clonar su repositorio de código. Si abrimos este fichero, podemos observar como se definen dos contenedores:
- ***server***: contenedor que contiene el *backend* o servidor de Unity Catalog. Habilita el puerto 8080 para poder acceder a los diferentes servicios disponibles (catálogos, tablas, volúmenes...). Comprenderemos esto más adelante.
- ***ui***: contenedor que levanta la interfaz de usuario, disponible en el puerto 3000.

![Docker - Compose YAML](./Imagenes/docker_compose-yaml.png)

Para iniciar nuestra aplicación ejecutamos el comando ```docker compose up``` el cual levanta los contenedores definidos en el fichero *compose.yaml* anteriormente analizado.

In [ ]:
%%sh
cd unitycatalog
docker compose up -d

En *Docker Desktop > Containers* podemos ahora observar los dos contenedores correctamente levantados y en ejecución. El servidor en el puerto 8080 y la interfaz de usuario en el puerto 3000.

![Docker - Containers 1](./Imagenes/docker_containers-1.png)

Si seleccionamos el contenedor *server-1*, en su sección *Logs*, podemos ver por pantalla el rótulo de Unity Catalog junto con la versión desplegada, en nuestro caso, v0.3.0.

![Docker - Server Logs](./Imagenes/docker_server-logs.png)

### Primeros pasos con Unity Catalog

Unity Catalog puede gobernar y gestionar cuatro tipos de recursos clave:
- **Tablas de datos**: Conjuntos de datos estructurados (columnas/filas) que se consultan y gobiernan de forma centralizada.
- **Volúmenes**: Espacios para archivos sin estructura (CSV, JSON, logs…), pero igualmente gobernados a nivel de permisos y trazabilidad.
- **Funciones**: Lógicas definidas para transformar o procesar datos que permiten la reutilización de código.
- **Modelos**: Artefactos de Machine Learning que pueden almacenarse y versionarse dentro de Unity Catalog para control y colaboración.

Estos recursos se pueden organizar en diferentes **catálogos** y **esquemas**, los cuales permiten establecer diferentes niveles jerárquicos de organización de recursos. Si, en nuestro PC, abrimos un navegador (i.e. Google Chrome) y navegamos a la URL http://localhost:3000, podremos ver a través del interfaz de usuario los catálogos y esquemas existentes.

![Unity Catalog - UI Start](./Imagenes/uc_ui-start.png)

Podemos observar como esta versión de Unity Catalog trae por defecto creado un catálogo llamado *unity* que alberga un esquema llamado *default* que a su vez alberga cuatro tablas de datos *marksheet*, *marksheet_uniform*, *numbers* y *user_countries*. Podemos navegar entre los diferentes recursos para familiarizarnos con este tipo de conceptos.

También podemos utilizar directamente la **API (*Application Programming Interface*)** expuesta por el backend de Unity Catalog para consultar esta misma información de forma programática. La documentación de esta API está disponible en [esta página](https://docs.unitycatalog.io/swagger-docs/). A modo de ejemplo, si introducimos en nuestro navegador la siguiente URL, estaremos realizando una operación GET (lectura) de las tablas disponibles en el catálogo *unity*, esquema *default*.

```http://localhost:8080/api/2.1/unity-catalog/tables?catalog_name=unity&schema_name=default```

![Unity Catalog - API Request](./Imagenes/uc_ui-api-request.png)

Alternativamente, Unity Catalog proporciona un **CLI (Command Line Interface)** que simplifica la ejecución de llamadas a la API desde la línea de comandos. Para no tener que instalar sus dependencias en nuestro propio PC, utilizaremos esta utilidad desde el contenedor del backend, donde ya está disponible. A modo de ejemplo, repetimos la misma operación que realizamos antes a través de la API, pero ahora con el CLI: listamos las tablas disponibles en el catálogo *unity*, dentro del esquema *default*.

In [ ]:
%%sh
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc table list --catalog unity --schema default"


El comando ```docker exec unitycatalog-server-1 /bin/bash``` permite ejecutar el proceso */bin/bash* en el contenedor docker llamado *unitycatalog-server-1*, es decir, ejecutar comandos directamente sobre la shell de contenedor. Precisamente la opción ```-c``` permite indicar el comando que queremos ejecutar, en nuestro caso```bin/uc table list --catalog unity --schema default```. Este comando ejecuta el CLI de Unity Catalog, disponible en la ruta ```bin/uc```, y le indica que desea listar las tablas del catálogo *unity* y esquema *default*.

Toda la documentación sobre cómo utilizar este CLI está disponible en la [página de documentación](https://docs.unitycatalog.io/usage/cli/) de Unity Catalog.

### Configuración Metastore en MySQL

Hasta ahora, hemos podido realizar un despliegue exitoso de la aplicación Unity Catalog y comprender tanto sus conceptos principales (catálogo, esquema, tabla, volumen...) como diferentes formas de interactuar y gestionar la aplicación (interfaz de usuario, API, CLI). No obstante, como indicamos al empezar este laboratorio, deseamos que el almacenamiento de los metadatos de los recursos gestionados por Unity Catalog se almacenen en una base de datos al uso en lugar de en el propio sistema de ficheros, que es la opción por defecto aplicada en el despliegue que acabamos de realizar.

Unity Catalog ofrece diferentes alternativas para almacenar los metadatos de los recursos que gobierna, como sistema de ficheros o base de datos. En nuestro caso, configuraremos una base de datos [MySQL](https://www.mysql.com/) como repositorio para el almacenamiento de estos metadatos.

Para ello realizaremos los siguientes pasos:
- Descargar un conector para que Unity Catalog pueda conectarse a una base de datos de tipo MySQL.
- Levantar un contenedor docker con la base de datos MySQL.
- Configurar en Unity Catalog los parámetros de dicha conexión.

#### Driver JDBC

Podemos obtener el conector (driver JDBC) en el [siguiente enlace](https://dev.mysql.com/downloads/connector/j/) seleccionando la opción *Platform Independent*.

![MySQL - JDBC Download](./Imagenes/mysql_jdbc-download.png)

Esto descargará un fichero *.zip* dentro del cual encontraremos el driver: fichero *.jar* que debemos copiar dentro de la carpeta *unitycatalog*. Debemos modificar su nombre a *mysql-connector-j-9.2.0.jar*, si no fuese el nombre que trajese por defecto el archivo descargado.

En el momento de la preparación de este tutorial, la última versión disponible es la *9.2.0*. En caso de que hayan sido publicadas nuevas versiones, en el momento de realizar la descarga podremos seleccionar esta versión desde la sección *Archives*.

![MySQL - JDBC Download Archive](./Imagenes/mysql_jdbc-download-archive.png)


#### Configuración y despliegue

En la carpeta MySQL econtramos varios archivos para poder realizar la configuración y el despliegue de la solución:
- Los archivos *compose-MySQL.yaml* y *Dockerfile-MySQL* modifican el anterior despliegue de contenedores realizado incluyendo un nuevo contenedor *db* que aloja la base de datos MySQL.
- El archivo *hibernate.properties* permite configurar en Unity Catalog los parámetros necesarios para conectar con MySQL. Entre ellos:
```
        # MySQL JDBC Driver
        hibernate.connection.driver_class=com.mysql.cj.jdbc.Driver
        # MySQL URL
        hibernate.connection.url=jdbc:mysql://db:3306/ucdb
        # MySQL User
        hibernate.connection.user=uc_default_user
        # MySQL Password              
        hibernate.connection.password=uc_default_password
 ```

El siguiente script permite realizar la copia de los archivos anteriormente indicados y el re-despliegue de contenedores.

In [ ]:
%%sh
cp MySQL/compose-MySQL.yaml MySQL/Dockerfile-MySQL unitycatalog/
cp MySQL/hibernate.properties unitycatalog/etc/conf/
cd unitycatalog
docker compose -f compose-MySQL.yaml up --build -d

#### Validación

En Docker Desktop podemos ahora ver un nuevo contenedor *db* como parte del despliegue de la aplicación.

![Docker - Containers MySQL](./Imagenes/docker_containers-2.png)

Al encontrarse la base de datos MySQL vacía, si refrescamos la web de Unity Catalog veremos como ahora no aparece ningún catálogo.

![Unity Catalog - UI MySQL](./Imagenes/uc_ui-mysql.png)

Lo mismo sucede si tratamos de listar los catálogos disponibles utilizando el CLI. Vemos un listado vacío ya que no existe ningún catálogo.

In [ ]:
%%sh
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc catalog list"

Por último, podemos acceder directamente a la base de datos MySQL mediante Python instalando la librería *mysql-connector*.

In [ ]:
%%sh
source python-venv/bin/activate
pip install mysql-connector-python

Una vez instalada, importamos la librería e introducimos los parámetros de conexión (URL, usuario, contraseña y nombre de la base de datos) de igual forma que hicimos anteriormente desde Unity Catalog. A continuación ejecutamos un primer comando SQL para mostrar las tablas disponibles en la base de datos: 

```SHOW TABLES```.

In [ ]:
import mysql.connector

# Conexión a la base de datos
conn = mysql.connector.connect(
    host="localhost",          # Dirección del servidor, en nuestro caso nuestro propio PC
    user="uc_default_user",         # Usuario de base de datos anteriormente configurado
    password="uc_default_password",  # Contraseña anteriormente configurada
    database="ucdb"  # Nombre de la base de datos
)

try:
    cursor = conn.cursor()

    # Mostrar tablas en la base de datos
    cursor.execute("SHOW TABLES")
    tablas = cursor.fetchall()
    print("Tablas en la base de datos:")
    for t in tablas:
        # Cada fila es una tupla con un solo valor (el nombre de la tabla)
        print(" -", t[0])

finally:
    # Cerrar cursor y conexión
    cursor.close()

En este caso, podemos observar las diferentes tablas que crea Unity Catalog en MySQL para almacenar los metadatos necesarios. Si, al igual que hicimos desde la web o el CLI, deseamos listar los catálogos disponibles, podemos ejecutar el siguiente comando SQL:

```SELECT NAME FROM uc_catalogs```

In [ ]:
try:
    cursor = conn.cursor()

    # Mostrar tablas en la base de datos
    cursor.execute("SELECT NAME FROM uc_catalogs")
    query = cursor.fetchall()
    print("Catálogos en la base de datos:")
    for t in query:
        # Cada fila es una tupla con un solo valor (el nombre de la tabla)
        print(" -", t[0])

finally:
    # Cerrar cursor y conexión
    cursor.close()

A modo de prueba, anticipándonos al siguiente apartado, vamos a crear los catálogos de datos de tres ciudades. Utilizamos para ello el CLI.

In [ ]:
%%sh
# Crear Catalogo
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc catalog create --name Toledo --comment 'Catálogo de datos de la ciudad de Toledo'"
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc catalog create --name Madrid --comment 'Catálogo de datos de la ciudad de Madrid'"
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc catalog create --name Sevilla --comment 'Catálogo de datos de la ciudad de Sevilla'"

Podemos ver estos tres esquemas disponibles a través del UI.

![Unity Catalog - UI Catalogs](./Imagenes/uc_ui-catalogs.png)

Si volvemos a realizar la consulta utilizando *mysql connector*, observaremos ahora los tres catálogos recién creados.

In [ ]:
try:
    cursor = conn.cursor()

    # Mostrar tablas en la base de datos
    cursor.execute("SELECT NAME FROM uc_catalogs")
    query = cursor.fetchall()
    print("Catálogos en la base de datos:")
    for t in query:
        # Cada fila es una tupla con un solo valor (el nombre de la tabla)
        print(" -", t[0])

finally:
    # Cerrar cursor y conexión
    cursor.close()
    conn.close()

Una vez terminadas las consultas, podemos cerrar la conexión anteriormente creada.

In [ ]:
conn.close()

### Construcción de objetos y recursos

En los anteriores apartados configuramos y desplegamos Unity Catalog por lo que ya disponemos de un entorno de trabajo sobre el cual empezar a organizar y gestionar nuestros recursos de datos e IA. En este apartado nos centraremos en crear y catalogar los diferentes recursos siguiendo 

Recordando el escenario de aplicación, vamos a tratar de representar un caso de uso real en el que un organismo desea habilitar varios catálogos de datos y recursos de datos relativos a servicios públicos ofrecidos por diferentes ciudades españolas. Entre estos servicios, podremos encontrar información relativa a alumbrado, transporte público o gestión de residuos. Para ello, decidimos establecer la siguiente jerarquía de recursos dentro de Unity Catalog

![UC - Organization](./Imagenes/uc_organization.png)

Dado que cada ciudad tiene sus particularidades en cuanto a servicios, formato de datos e incluso políticas de publicación y compartición de datos, cada ciudad dispondrá de su propio catálogo de recursos. Posteriormente, dado que diferentes instituciones públicas gestionan cada uno de los servicios dentro de una misma ciudad, cada servicio dispondrá de su propio esquema donde gestionar sus recursos de datos. De esta forma, una ingeniera de datos realizando trabajos sobre datos de transporte público no tendrá por qué disponer de acceso a los recursos de alumbrado

Los pasos que daremos a continuación serán los siguientes:
- Creación de catálogos y esquemas.
- Creación de tablas y volúmenes de datos.
- Creación de funciones y modelos.

#### Creación de catálogos y esquemas

En el anterior apartado ya creamos los catálogos de tres ciudades para validar el correcto funcionamiento de la base de datos por lo que procederemos directamente a crear los esquemas correspondientes para la ciudad de Toledo. 

En nuestro camino para seguir conociendo Unity Catalog y sus capacidades utilizaremos [Apache Spark](https://spark.apache.org/), y en particular [Spark SQL](https://spark.apache.org/sql/), para completar los siguientes pasos. Unity Catalog dispone de capacidad de [integración](https://docs.unitycatalog.io/integrations/unity-catalog-spark/) con este motor de transformación y análisis de datos ampliamente utilizado por la comunidad de ingeniería y ciencia de datos.

Para poder interactuar con nuestro Unity Catalog a través de Spark SQL debemos disponer de JDK en nuestro PC (ya instalado en la etapa de *Requisitos Previos*) e instalar la librería python PySpark.

Dado que ya disponemos de un entorno python virtual previamente creado, la instalación de PySpark será muy sencilla:

In [ ]:
%%sh
# Instalamos PySpark
source python-venv/bin/activate
pip install pyspark

Con ambos elementos ya instalados, procedemos a crear una aplicación o sesión spark contra la que lanzaremos todos los trabajos que vayamos necesitando (cada una de los comandos SQL para crear elementos como esquemas o tablas, o realizar manipulaciones de datos). No profundizamos aquí en los detalles de este comando más allá de entender que estamos creando una aplicación Spark denominada *uc-sparksql* que trabajará sobre el catálogo *Toledo* accesible en la dirección *http://localhost:8080*, que es donde tenemos habilitado Unity Catalog.

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .appName("uc-sparksql")
        .master("local[*]")
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.2.1,io.unitycatalog:unitycatalog-spark_2.12:0.2.0")
        .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "io.unitycatalog.spark.UCSingleCatalog")
        .config("spark.sql.catalog.Toledo", "io.unitycatalog.spark.UCSingleCatalog")
        .config("spark.sql.catalog.Toledo.uri", "http://localhost:8080")
        .config("spark.sql.catalog.Toledo.token", "")
        .config("spark.sql.defaultCatalog", "Toledo")
        .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

A continuación, creamos en el catálogo *Toledo* los esquemas para los servicios *transporte público*, *gestión de residuos* y *alumbrado*.

In [ ]:
# Creamos esquemas de diferentes servicios
spark.sql("CREATE SCHEMA IF NOT EXISTS transporte_publico")
spark.sql("CREATE SCHEMA IF NOT EXISTS gestion_residuos")
spark.sql("CREATE SCHEMA IF NOT EXISTS alumbrado")

# Mostramos los esquemas creados
spark.sql("SHOW SCHEMAS").show()

Podemos observar estos esquemas recién creados en el UI de Unity Catalog.

![Unity Catalog - UI Schemas](./Imagenes/uc_ui-schemas.png)

Vemos como gracias a esta integración podemos trabajar a través de consultas SQL directamente sobre nuestro entorno Unity Catalog.

#### Creación de tablas y volúmenes

A continuación vamos a proceder a crear varias tablas y un volumen dentro de el esquema de transporte público. En primer lugar, suponemos que el servicio de transporte público dispone de las siguientes tablas:

- **rutas**: Representa las líneas o recorridos que realizan los autobuses. Campos:
    - ruta_id (PK): Identificador único de la ruta (entero).
    - nombre: Nombre o código de la ruta (cadena).

- **paradas**: Representa las paradas de autobús. Campos:
    - parada_id (PK): Identificador único de la parada (entero).
    - nombre: Nombre de la parada (cadena).
    - lat, lon: Coordenadas de latitud y longitud (decimal). Permiten geolocalizar la parada en el mapa.

- **rutas_paradas**: Relación entre rutas y paradas.
    - ruta_id (FK): Hace referencia a rutas.ruta_id.
    - parada_id (FK): Hace referencia a paradas.parada_id.
    - parada_orden: Entero que define la secuencia en la que la ruta atiende cada parada (p.ej. 1, 2, 3…).

- **autobuses**: Cada uno de los autobuses de la flota. Campos:
    - bus_id (PK): Identificador único de autobús (entero).
    - matricula: Matrícula del autobús (cadena).
    - modelo, marca: Modelo y marca (cadenas).
    - capacidad: Capacidad de pasajeros (entero).
    - ruta_id (FK opcional): Ruta a la que está asociado.

Ahora bien, estas tablas junto con sus datos deben estar alojadas en algún repositorio de datos. Como indicamos en el apartado de arquitectura, vamos a habilitar un bucket S3 en AWS donde poder alojar estos datos.

##### AWS como repositorio de datos

Para ello, en primer lugar, creamos una cuenta gratuita en AWS navegando a su [página oficial](https://aws.amazon.com/) y haciendo click en *Cree una cuenta en AWS* en la parte superior derecha. Esto nos llevará a una página de registro donde seguiremos varios pasos indicando nuestra dirección de correo electrónico, nuestra nueva contraseña y nuestros datos personales (nombre, teléfono, dirección...). 

A pesar de que vamos a utilizar los servicios gratuitos de AWS (*Free Tier*), durante el proceso AWS nos solicitará una tarjeta de crédito para poder confirmar la creación de la cuenta.

En la página de nivel de soporte, seleccionaremos el soporte básico gratuito.

![AWS - Create account](./Imagenes/aws_create-account.png)

Una vez creada la cuenta, volveremos a la [página oficial](https://aws.amazon.com/) de AWS y seleccionaremos esta vez *Iniciar sesión* e introduciremos nuestros datos de inicio de sesión marcando la opción **usuario raíz**.

![AWS - Login](./Imagenes/aws_login.png)

Una vez en nuestra cuenta, seleccionamos primero la región de trabajo *Europe - Ireland (eu-west-1)* en la esquina superior derecha. Esto indica que los servicios que creemos en nuestra cuenta de AWS se generarán en el conjunto de centros de procesamientos de datos que Amazon ha construido en Irlanda. Posteriormente, en el buscador situado en la parte superior de la pantalla, escribimos *S3* y seleccionamos el servicio correspondiente para acceder al mismo.

![AWS - First Steps](./Imagenes/aws_first-steps.png)

Una vez en S3, seleccionamos la opción *Create Bucket*, marcamos la opción *General purpose bucket* e introducimos el nombre del mismo. Lo llamaremos *toledo-{varios caracteres aleatorios}* debido a que contendrá los datos de esta ciudad y a que cada bucket debe tener un nombre único en todo AWS. Dejamos el resto de parámetros por defecto y seleccionamos de nuevo *Create Bucket* abajo en la parte final del formulario.

![AWS - Create Bucket](./Imagenes/aws_create-bucket.png)

Podemos ver nuestro bucket recientemente creado en el servicio S3.

![AWS - Bucket Created](./Imagenes/aws_bucket-created.png)

Si hacemos click en el nombre del bucket podemos acceder al contenido del mismo. Al entrar, dado que acaba de ser creado, veremos que se encuentra vacío, indicando *Objects(0)* en su parte superior.

![AWS - Bucket Objects](./Imagenes/aws_bucket-objects.png)

Para que Unity Catalog pueda crear objetos en este bucket, seguiremos los siguientes pasos:
- **PASO 1:** Crear un usuario *unity-catalog* en AWS y sus credenciales de acceso (AccessKey y SecretKey).
- **PASO 2:** Crear una política con permisos suficientes para que Unity Catalog pueda realizar las acciones necesarias sobre el bucket.
- **PASO 3:** Crear un rol que pueda ser asumido por el usuario *unity-catalog* y disponga de los permisos definidos en la anterior política.
- **PASO 4:** Configurar los parámetros de conexión en Unity Catalog.

**PASO 1:**

Buscamos y accedemos al servicio AWS IAM (Identity and Access Management) buscándolo en el catálogo de servicios de igual forma que hicimos con AWS S3. Una vez en su pantalla principal, seleccionamos la opción *Users* en el panel izquierdo y *Create User*.

![AWS - User Create](./Imagenes/aws_user-create.png)

A continuación introducimos su nombre, *unity-catalog*, y pulsamos varias veces en *Next* y finalmente en *Create User* dejando la configuración por defecto.

Seleccionamos desde la pantalla *Users* el usuario *unity-catalog* recientemente creado. Desde la pantalla detalle del usuario:
1. Copiamos su identificador único o ARN (Amazon Resource name) y lo guardamos para más tarde.
2. Navegamos a la sección *Security credentials* y seleccionamos la opción *Create access key*.

![AWS - User Credentials](./Imagenes/aws_user-credentials.png)

En la sección *Use Case* marcamos la opción *Other* y hacemos clic en *Next*. Posteriormente seleccionamos la opción *Create access key* para terminar el proceso de creación. Antes de abandonar esta pantalla, seleccionamos la opción *Donwload .csv* para descargar el fichero que contiene las credenciales de nuestro usuario *unity-catalog*.

![AWS - User Credentials](./Imagenes/aws_user-credentials-download.png)

**PASO 2:**

Ya disponemos de un usuario y de sus credenciales de acceso, ahora debemos garantizar que este usuario dispone de los permisos adecuados para operar con nuestro bucket. Para ello, desde la pantalla principal de IAM, seleccionamos *Policies* en el panel lateral izquierdo y después *Create policy*.

A continuación, seleccionamos la edición JSON, en el *Policy editor* y copiamos el siguiente texto que permite acciones clave como crear y obtener objectos de nuestro bucket. **Importante:** En el texto JSON copiado deberás sustituir el nombre del bucket por el del tuyo propio. Después pulsa *Next*.

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket",
                "s3:GetBucketLocation"
            ],
            "Resource": "arn:aws:s3:::toledo-jk345"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:DeleteObject",
                "s3:AbortMultipartUpload"
            ],
            "Resource": "arn:aws:s3:::toledo-jk345/*"
        }
    ]
}
``````

![AWS - Policy Create](./Imagenes/aws_policy-create.png)

A continuación, especifíca el nombre de la nueva política, *unity-catalog-policy*, y pulsa en *Create policy* para terminar el proceso de creación.

**PASO 3:**

Desde el panel izquierdo de IAM accede a *Roles* y selecciona *Create role*. En el apartado *Trusted entity type* selecciona *Custom trust policy*.

En el editor abajo, deberás indicar ``"AWS":"arn:aws:iam::123456789:user/unity-catalog"``, sustituyendo este ARN por el ARN de tu usuario anteriormente copiado. Lo que le estamos indicando a AWS es que nuestro usuario *unity-catalog* puede asumir el rol que estamos creando.

![AWS - Policy Create](./Imagenes/aws_role-create.png)


Pulsa *Next* para pasar a la asignación de permisos. Ahora busca la política anteriormente creada, *unity-catalog-policy*, selecciónala y pulsa *Next*.
Por último, utiliza *unity-catalog-role* como nombre del nuevo rol y selecciona *Create role* para terminar el proceso.


![AWS - Policy Create](./Imagenes/aws_role-add-permission.png)

Al igual que antes hicimos con nuestro usuario, debemos ahora guardar el ARN del rol creado. Desde la sección *Roles*, selecciona el rol *unity-catalog-role* recién creado y en su página de detalle copia el ARN del mismo.

**PASO 4**

Tras realizar todas las configuraciones necesarias en AWS, es ahora momento de indicarle a Unity Catalog cómo encontrar y trabajar con nuestro bucket de S3. Para ello, debemos parametrizar el fichero *server.properties* que encontrarás en la carpeta *unitycatalog/etc/conf*.

Abrimos el fichero con Visual Studio Code (o cualquier otro editor de ficheros de texto) y modificamos los siguientes parámetros de configuración de Unity Catalog:
```
s3.bucketPath.0=s3://toledo-jk345
s3.region.0=eu-west-1
s3.awsRoleArn.0=arn:aws:iam::123456789:role/unity-catalog-role
s3.accessKey.0=AKIA3...
s3.secretKey.0=JIIPWoG...
``````

Debes tener en cuenta que:
- Debes modificar el *bucketPath* indicando el nombre de tu propio bucket.
- Debes copiar el ARN de tu rol en *awsRoleArn*.
- Encontrarás tu *accessKey* y *secretKey* en el fichero CSV que descargamos cuando configuramos las credenciales de nuestro usuario *unity-catalog*.

Una vez modificado el archivo y guardado, debemos reiniciar el contenedor para que aplique la nueva configuración.

In [ ]:
%%sh
docker restart unitycatalog-server-1

##### Creación de tablas

Vamos a crear ahora cada una de las tablas anteriormente definidas. Para ello, ejecutamos el siguiente código PySpark sustituyendo el valor de la variable *bucket_name* por el nombre de nuestro propio bucket.

In [ ]:
bucket_name = "toledo-jk345"

spark.sql(f"""
CREATE TABLE IF NOT EXISTS transporte_publico.rutas (
    ruta_id INT,
    nombre STRING
)
USING DELTA
LOCATION 's3://{bucket_name}/transporte_publico/tablas/rutas'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS transporte_publico.paradas (
    parada_id INT,
    nombre STRING,
    lat DECIMAL(8,6),
    lon DECIMAL(9,6)
)
USING DELTA
LOCATION 's3://{bucket_name}/transporte_publico/tablas/paradas'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS transporte_publico.rutas_paradas (
    ruta_id INT,
    parada_id INT,
    parada_orden INT
)
USING DELTA
LOCATION 's3://{bucket_name}/transporte_publico/tablas/rutas_paradas'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS transporte_publico.autobuses (
    bus_id INT,
    matricula STRING,
    modelo STRING,
    marca STRING,
    capacidad INT,
    ruta_id INT
)
USING DELTA
LOCATION 's3://{bucket_name}/transporte_publico/tablas/autobuses'
""")

Ya podemos ver las tablas creadas en el UI de Unity Catalog, pero aún no observaremos la creación de las mismas en nuestro bucket, ya que por ahora únicamente hemos creado la definición de las mismas y no tienen ningún contenido.

Cargamos ahora algunos datos de prueba en la tabla autobuses.

In [ ]:
spark.sql("INSERT INTO Toledo.transporte_publico.autobuses VALUES (1, '4560KLN', 'E-City L12', 'ALFABUS', 88, 91)")
spark.sql("INSERT INTO Toledo.transporte_publico.autobuses VALUES (2, '5013KLJ', 'E-City L12', 'ALFABUS', 88, 91)")
spark.sql("INSERT INTO Toledo.transporte_publico.autobuses VALUES (3, '9990JOD', 'E-City L12', 'ALFABUS', 88, 92)")
spark.sql("INSERT INTO Toledo.transporte_publico.autobuses VALUES (4, '4011LLD', 'i3', 'IRIZAR', 76, 92)")
spark.sql("INSERT INTO Toledo.transporte_publico.autobuses VALUES (5, '8800LJK', 'i3', 'IRIZAR', 76, 14)")
spark.sql("INSERT INTO Toledo.transporte_publico.autobuses VALUES (6, '1002HVS', 'i3', 'IRIZAR', 76, 14)")

Vemos como efectivamente estos datos han sido correctamente almacenados en la tabla.

In [ ]:
spark.sql("SELECT * FROM Toledo.transporte_publico.autobuses").show()

Si ahora consultamos el contenido de nuestro bucket en AWS, observaremos lo siguiente. (TODO)

##### Creación de volumen

Nuestro caso de uso de transporte público requiere el almacenamiento de los datos de telemetría enviados por los autobuses. Se trata de eventos generados periódicamente por los autobuses que miden aspectos como la ubicación del autobús, el número de pasajeros o el nivel de llenado de combustible (% de su depósito o de su batería, en el caso de eléctricos). Estos eventos se registran en ficheros JSON almacenados en crudo en el bucket de AWS.

Para ello, en primer lugar creamos el volumen deseado en Unity Catalog utilizando el CLI. Recuerda sustituir el nombre del bucket por el tuyo propio.

In [ ]:
%%sh
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc volume create --full_name Toledo.transporte_publico.telemetria --storage_location 's3://toledo-jk345/transporte_publico/volumenes/telemetria'"

A continuación, cargamos varios ficheros de ejemplo en el bucket de S3. Para ello, navegamos hasta la ruta *transporte_publico* de nuestro bucket S3 en AWS, y seleccionamos la opción *Upload*.

![AWS - Create Volume](./Imagenes/aws_volume-create.png)

Ahora, seleccionamos la opción *Add folder* y seleccionamos la carpeta *volumenes* que encontramos dentro de la carpeta *Telemetria* en nuestra carpeta de trabajo. Veremos como se han creado cuatro nuevos ficheros JSON en el volumen *telemetria*.

![AWS - Volume Load Files](./Imagenes/aws_volume-load-files.png)

Podemos ahora listar los ficheros disponibles dentro de este volumen a través del CLI.

In [ ]:
%%sh
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc volume read --full_name Toledo.transporte_publico.telemetria"

Y mostrar el contenido de cada uno de ellos indicando el nombre del fichero correspondiente.

In [ ]:
%%sh
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc volume read --full_name Toledo.transporte_publico.telemetria --path 3_3aef8e84-6b18-42fa-9160-3979f9f6b0ff_1740554060000.json"

#### Creación de funciones y modelos

Hasta este momento, hemos creado diferentes recursos de datos como tablas y volúmenes gracias a Unity Catalog. En este apartado, pasaremos a registrar otro tipo de recursos como funciones, que nos permiten la reutilización de código, o modelos de *machine learning*.

##### Funciones

Una cálculo de uso recurrente en el mundo del transporte público es el cálculo de distancias entre diferentes ubicaciones. En este apartado crearemos una función python capaz de calcular la distancia en línea recta entre dos pares de coordenadas geográficas y la registraremos en nuestro catálogo.

A continuación, escribimos la función para poder entender su comportamiento.
 
*Nota: En la versión actual de Unity Catalog no podemos indicar parámetros de tipo FLOAT en sus funciones, de ahí pasar las coordenadas como enteros.*

In [ ]:
def haversine_distance(lat1_deg, lon1_deg, lat2_deg, lon2_deg):
    import math
    R = 6371.0  # Radio de la Tierra en km

    # Convertir grados a radianes
    lat1 = math.radians(lat1_deg/10000)
    lon1 = math.radians(lon1_deg/10000)
    lat2 = math.radians(lat2_deg/10000)
    lon2 = math.radians(lon2_deg/10000)

    # Diferencias
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula del haversine
    a = (math.sin(dlat / 2)**2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2)
    c = 2 * math.asin(math.sqrt(a))

    distance = round(R * c,3)
    return distance

print(f"{haversine_distance(398580, -40203, 398592, -40323)} km") # Distancia entre el Alcázar de Toledo y la Puerta del Cambrón

Ahora, vamos a usar el CLI para registrar esta función en nuestro esquema de transporte público:

In [ ]:
%%sh
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc function create --full_name Toledo.transporte_publico.distancia_haversine --data_type FLOAT --input_params 'lat1_deg int, lon1_deg int, lat2_deg int, lon2_deg int' --language 'python' --def 'import math\nR = 6371.0  # Radio de la Tierra en km\n\n# Convertir grados a radianes\nlat1 = math.radians(lat1_deg/10000)\nlon1 = math.radians(lon1_deg/10000)\nlat2 = math.radians(lat2_deg/10000)\nlon2 = math.radians(lon2_deg/10000)\n\n# Diferencias\ndlat = lat2 - lat1\ndlon = lon2 - lon1\n\n# Fórmula del haversine\na = (math.sin(dlat / 2)**2\n     + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2)\nc = 2 * math.asin(math.sqrt(a))\n\ndistance = round(R * c,3)\nreturn distance'"

Podemos ver cómo la función ha sido correctamente registrada en Unity Catalog.

![Unity Catalog - Function](./Imagenes/uc_function.png)

Además, podemos ejecutarla invocándola desde Unity Catalog con los parámetros correspondientes:

In [ ]:
%%sh
docker exec unitycatalog-server-1 /bin/bash -c "bin/uc function call --full_name Toledo.transporte_publico.distancia_haversine --input_params '398580, -40203, 398592, -40323'"

##### Modelos de machine learning

El último tipo de recurso que vamos a aprender a gestionar desde Unity Catalog es altamente relevante: modelos de machine learning.

Para ello, primero instalaremos una serie de librerías que serán necesarias: *mlflow*, para el registro del modelo, y *pandas* y *scikit-learn*, para la construcción del modelo.

In [ ]:
%%sh
source python-venv/bin/activate
pip install mlflow
pip install pandas
pip install scikit-learn

A continuación, arrancamos un servidor local de *mlflow* en nuestro PC ejecutando los siguientes comandos en un terminal abierto en nuestra carpeta de trabajo.

```
source python-venv/bin/activate
mlflow ui
```

![MLFlow - Start](./Imagenes/mlflow_start.png)

Ahora, crearemos un modelo de ejemplo (no tienen ninguna relación con nuestro caso de uso de transporte público) y lo registraremos en nuestro catálogo.

In [ ]:
import mlflow
import os
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

#Parametrizamos direcciones de servidor MlFlow y servidor Unity Catalog
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_registry_uri("uc:http://127.0.0.1:8080")

# Generamos nuestro modelo de machine learning, en este caso, de tipo RandomForestClassifier, y lo registramos
X, y = datasets.load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamos el modelo y lo registramos en el catálogo, suponemos que predice el tiempo hasta la siguiente parada
with mlflow.start_run():
    clf = RandomForestClassifier(max_depth=7)
    clf.fit(X_train, y_train)
    input_example = X_train.iloc[[0]]
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="model",
        input_example=input_example,
        registered_model_name="Toledo.transporte_publico.prediccion_tiempo_siguiente_parada",
    )

Podemos ver como la versión 1 de este modelo ha sido correctamente catalogada.

![Unity Catalog - Model](./Imagenes/uc_model.png)

Ahora, invocamos el modelo ya registrado en Unity Catalog y realizamos una predicción.

In [ ]:
loaded_model = mlflow.pyfunc.load_model(f"models:/Toledo.transporte_publico.prediccion_tiempo_siguiente_parada/1")
predictions = loaded_model.predict(X_test)
iris_feature_names = datasets.load_iris().feature_names
result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions
result[:4]

## Resultados y conclusiones

Como resultado de este laboratorio prático, hemos podido conocer la herramienta Unity Catalog como plataforma abierta para la gobernanza de datos y recursos de datos como modelos de *machine learning*. Hemos podido conocer en el contexto de un caso de uso concreto y con un ecosistema de herramientas similar al que podemos encontrar en una organización real, sus capacidades, su modo de despliegue y su uso.

Veremos en los próximos años si este tipo de herramientas alcanzan el nivel de estandarización necesario para transformar la forma en que se administran y comparten los recursos de datos en múltiples sectores.

## Posibles Definiciones a crear
- Bucket S3:
- Metastore:
- Shell /bin/bash:
- Spark:
- Spark SQL:
- MLFlow